In [4]:
import numpy as np
import pandas as pd

In [5]:
"""
Looking at population density and distribution of people

Dataset:
Base Zoning: http://opendata.columbus.gov/datasets/96f7642a62f84db997f9e1db4a776995_4
    - Can look at zoning to see which locations are more populated
    
BZA Zoning Variances: http://opendata.columbus.gov/datasets/19786dd084e644a4aea6b33f867dd631_1
    - More Zoning?
    
Recommended Land Use: http://opendata.columbus.gov/datasets/26f0606f94db4c07a63aef3cc8927c9b_21
    - Where we can build charging stations?
    
Population Density Map: https://apps.morpc.org/census2010/


Webscraping data:
    - https://github.com/ikhlaqsidhu/data-x/blob/master/03-tools-webscraping-crawling_api_afo/notebook-webscraping_v4.ipynb

API to convert Lat/Long to census block
    - https://geo.fcc.gov/api/census/#!/area/get_area
    
Conversion for GEOIDs
    - https://www.census.gov/geo/reference/codes/cou.html
    - https://www.census.gov/geo/reference/geoidentifiers.html (General info on how it's generated)

# TODO: find a way to associate GEOID with latitude/longitude, somehow associate with city/county?

""";

In [6]:
%ls

Data/                   PT_data.ipynb           README.md
NS_data.ipynb           Points_of_Interest.csv


In [112]:
census_data = pd.read_excel("Data/Columbus_Population.xlsx")

In [113]:
print(len(census_data))
census_data.head()

1000


,GEOID,2010 Total Population,2010 White Population,2010 African American Population,2010 Asian Population,2010 Other Races Population,2010 Hispanic Population,2010 Total Households,2010 Occupied Households,2010 Vacant Households,2000 Total Population,2000 White Population,2000 African American Population,2000 Asian Population,2000 Other Races Population,2000 Total Housing Units,2000 Occupied Housing Units,2000 Vacant Housing Units,2000 Hispanic Population
0,390450303001,1124,1099,4,5,16,3,447,417,30,1103,1092,3,1,7,426,408,18,5
1,390450303002,1212,1175,3,6,28,7,565,536,29,1073,1053,1,7,12,498,479,19,6
2,390450303003,1019,999,1,5,14,7,419,393,26,1005,1001,0,0,4,396,377,19,2
3,390450304001,1916,1869,11,4,32,14,735,709,26,1666,1627,7,3,29,600,579,21,23
4,390450304002,767,735,14,2,16,7,303,286,17,759,746,4,0,9,285,278,7,6


In [117]:
geoid_conversion = pd.read_csv("Data/OhioGEOIDConversion.csv").iloc[:,:4]
geoid_conversion.head()

,State,StateCode,CountyCode,CountyName
0,OH,39,1,Adams County
1,OH,39,3,Allen County
2,OH,39,5,Ashland County
3,OH,39,7,Ashtabula County
4,OH,39,9,Athens County


In [125]:
def convertGeoidToCounty(geoid):
    countyCode = int((geoid - 390000000000) / 10000000)
    return geoid_conversion[geoid_conversion["CountyCode"] == countyCode].CountyName.item()

geoids = census_data.iloc[:,0]
countyNames = geoids.apply(convertGeoidToCounty)
census_data.insert(1, "CountyNames", countyNames)
census_data.head()

,GEOID,CountyNames,2010 Total Population,2010 White Population,2010 African American Population,2010 Asian Population,2010 Other Races Population,2010 Hispanic Population,2010 Total Households,2010 Occupied Households,2010 Vacant Households,2000 Total Population,2000 White Population,2000 African American Population,2000 Asian Population,2000 Other Races Population,2000 Total Housing Units,2000 Occupied Housing Units,2000 Vacant Housing Units,2000 Hispanic Population
0,390450303001,Fairfield County,1124,1099,4,5,16,3,447,417,30,1103,1092,3,1,7,426,408,18,5
1,390450303002,Fairfield County,1212,1175,3,6,28,7,565,536,29,1073,1053,1,7,12,498,479,19,6
2,390450303003,Fairfield County,1019,999,1,5,14,7,419,393,26,1005,1001,0,0,4,396,377,19,2
3,390450304001,Fairfield County,1916,1869,11,4,32,14,735,709,26,1666,1627,7,3,29,600,579,21,23
4,390450304002,Fairfield County,767,735,14,2,16,7,303,286,17,759,746,4,0,9,285,278,7,6
